# FarmTech Solutions - Dashboard de Sensores Agrícolas

Este notebook apresenta uma visualização interativa dos dados sintéticos de sensores agrícolas, incluindo a lógica de atuação da bomba e sugestões de irrigação baseadas em clima simulado.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
# 1. Leitura e Preparação dos Dados
# O arquivo CSV gerado anteriormente é 'farmtech_sensor_data_multi_cultura.csv'
try:
    df = pd.read_csv('../document/farmtech_sensor_data_multi_cultura.csv', sep=';')
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
except FileNotFoundError:
    print("Erro: Arquivo 'farmtech_sensor_data_multi_cultura.csv' não encontrado. Certifique-se de que o arquivo CSV foi gerado.")
    # Cria um DataFrame vazio para evitar erros no restante do script
    df = pd.DataFrame()

# Parâmetros ideais (repetidos para a lógica de sugestão)
CULTURAS_PARAMETROS = {
    "Tomate": {"ph_min": 6.0, "ph_max": 7.0, "temp_min": 20.0, "temp_max": 28.0, "umid_solo_min": 60, "umid_solo_max": 80},
    "Cana-de-Açúcar": {"ph_min": 6.0, "ph_max": 6.5, "temp_min": 22.0, "temp_max": 30.0, "umid_solo_min": 50, "umid_solo_max": 70},
    "Laranja": {"ph_min": 5.5, "ph_max": 6.5, "temp_min": 23.0, "temp_max": 32.0, "umid_solo_min": 50, "umid_solo_max": 70},
    "Milho": {"ph_min": 5.5, "ph_max": 7.0, "temp_min": 24.0, "temp_max": 30.0, "umid_solo_min": 60, "umid_solo_max": 85},
    "Soja": {"ph_min": 6.0, "ph_max": 7.0, "temp_min": 20.0, "temp_max": 30.0, "umid_solo_min": 60, "umid_solo_max": 80},
    "Café": {"ph_min": 6.0, "ph_max": 6.5, "temp_min": 18.0, "temp_max": 22.0, "umid_solo_min": 50, "umid_solo_max": 70}
}

In [3]:
# 2. Lógica de Sugestão de Irrigação Baseada em Clima (Simulada)
def gerar_sugestao_irrigacao(row):
    cultura = row['CULTURA']
    params = CULTURAS_PARAMETROS.get(cultura, {})
    
    # Condições de Sensores
    umid_solo = row['UMIDADE_SOLO_PERC']
    temp_ar = row['TEMPERATURA_AR_C']
    ph_solo = row['PH_SOLO']
    
    # Condições de Clima (Simuladas a partir da Temperatura do Ar)
    # Simulação: Clima quente e seco (Temp > 30C e Umid Ar < 50%) aumenta a necessidade
    clima_quente_seco = (temp_ar > 30) and (row['UMIDADE_AR_PERC'] < 50)
    
    # Sugestão 1: Irrigação Necessária (Solo Seco ou Condição Crítica)
    if umid_solo < params.get("umid_solo_min", 60) * 0.8: # 80% do mínimo ideal
        return "IRRIGAR AGORA (Solo Seco)"
    
    # Sugestão 2: Irrigação Preventiva (Solo abaixo do ideal + Clima Desfavorável)
    if umid_solo < params.get("umid_solo_min", 60) and clima_quente_seco:
        return "IRRIGAR PREVENTIVAMENTE (Clima Quente/Seco)"
        
    # Sugestão 3: Irrigação Suspensa (Solo Saturado)
    if umid_solo > params.get("umid_solo_max", 80) * 1.1: # 110% do máximo ideal
        return "SUSPENDER IRRIGACAO (Solo Saturado)"
        
    # Sugestão 4: Condição Ideal
    if umid_solo >= params.get("umid_solo_min", 60) and umid_solo <= params.get("umid_solo_max", 80):
        return "CONDICAO IDEAL"
        
    return "MONITORAR"

if not df.empty:
    df['SUGESTAO_IRRIGACAO'] = df.apply(gerar_sugestao_irrigacao, axis=1)

In [4]:
# 3. Funções de Visualização com Plotly

def plot_sensor_time_series(df_filtered, sensor_col, title):
    """Gera um gráfico de linha para um sensor ao longo do tempo."""
    fig = px.line(df_filtered, x='TIMESTAMP', y=sensor_col, color='CULTURA',
                  title=title, template='plotly_white')
    fig.update_layout(xaxis_title="Data e Hora", yaxis_title=sensor_col)
    return fig

def plot_nutrients_status(df_filtered):
    """Gera um gráfico de barras para o status de nutrientes (N, P, K)."""
    nutrients_data = df_filtered[['N_PRESENTE', 'P_PRESENTE', 'K_PRESENTE']].sum().reset_index()
    nutrients_data.columns = ['Nutriente', 'Contagem']
    
    fig = px.bar(nutrients_data, x='Nutriente', y='Contagem', 
                 title='Contagem de Presença de Nutrientes (N, P, K)',
                 color='Nutriente', template='plotly_white')
    return fig

def plot_irrigacao_status(df_filtered):
    """Gera um gráfico de pizza para o status da bomba e sugestão de irrigação."""
    
    # Status da Bomba (Atuação Real)
    bomba_counts = df_filtered['BOMBA_LIGADA'].map({1: 'Ligada', 0: 'Desligada'}).value_counts().reset_index()
    bomba_counts.columns = ['Status', 'Contagem']
    fig_bomba = px.pie(bomba_counts, values='Contagem', names='Status', 
                       title='Status de Atuação da Bomba (BOMBA_LIGADA)',
                       color_discrete_sequence=px.colors.qualitative.Safe)
                       
    # Sugestão de Irrigação (Análise)
    sugestao_counts = df_filtered['SUGESTAO_IRRIGACAO'].value_counts().reset_index()
    sugestao_counts.columns = ['Sugestão', 'Contagem']
    fig_sugestao = px.pie(sugestao_counts, values='Contagem', names='Sugestão', 
                          title='Sugestão de Irrigação Baseada em Clima e Sensores',
                          color_discrete_sequence=px.colors.qualitative.Pastel)
                          
    return fig_bomba, fig_sugestao

In [6]:
# 4. Estrutura do Dashboard (Exibição no Jupyter Notebook)
def exibir_dashboard(df):
    if df.empty:
        return
        
    # Filtra para a cultura mais recente (última no CSV) para as visualizações iniciais
    cultura_selecionada = df['CULTURA'].iloc[-1]
    df_filtered = df[df['CULTURA'] == cultura_selecionada]
    
    print(f"## Dashboard de Monitoramento Agrícola - Cultura: {cultura_selecionada}")
    print(f"### Análise de Séries Temporais para {cultura_selecionada}")
    
    # Gráfico 1: Umidade do Solo
    fig_umidade = plot_sensor_time_series(df_filtered, 'UMIDADE_SOLO_PERC', 'Umidade do Solo (%) ao Longo do Tempo')
    fig_umidade.show()
    
    # Gráfico 2: pH do Solo
    fig_ph = plot_sensor_time_series(df_filtered, 'PH_SOLO', 'pH do Solo ao Longo do Tempo')
    fig_ph.show()
    
    # Gráfico 3: Temperatura do Ar
    fig_temp = plot_sensor_time_series(df_filtered, 'TEMPERATURA_AR_C', 'Temperatura do Ar (°C) ao Longo do Tempo')
    fig_temp.show()
    
    print(f"### Status de Nutrientes e Irrigação para {cultura_selecionada}")
    
    # Gráfico 4: Status de Nutrientes
    fig_nutrientes = plot_nutrients_status(df_filtered)
    fig_nutrientes.show()
    
    # Gráfico 5 e 6: Status da Bomba e Sugestão de Irrigação
    fig_bomba, fig_sugestao = plot_irrigacao_status(df_filtered)
    fig_bomba.show()
    fig_sugestao.show()
    
    print("---")
    print("## Tabela de Dados Brutos (Amostra)")
    # Exibe uma amostra dos dados
    from IPython.display import display, Markdown
    display(Markdown(df_filtered.tail(10).to_markdown(index=False)))


In [7]:
# Execução
if not df.empty:
    exibir_dashboard(df)
else:
    print("Não foi possível gerar o Dashboard devido à falta de dados.")

# Nota: Em um ambiente Jupyter real, o código seria dividido em células e a função .show() 
# do Plotly renderizaria os gráficos interativos.

## Dashboard de Monitoramento Agrícola - Cultura: Café
### Análise de Séries Temporais para Café


### Status de Nutrientes e Irrigação para Café


---
## Tabela de Dados Brutos (Amostra)


| TIMESTAMP           | ID_SENSOR     | CULTURA   | FASE_CULTURA          |   TEMPERATURA_AR_C |   UMIDADE_AR_PERC |   PH_SOLO |   UMIDADE_SOLO_PERC |   N_PRESENTE |   P_PRESENTE |   K_PRESENTE |   BOMBA_LIGADA | SUGESTAO_IRRIGACAO   |
|:--------------------|:--------------|:----------|:----------------------|-------------------:|------------------:|----------:|--------------------:|-------------:|-------------:|-------------:|---------------:|:---------------------|
| 2025-10-21 18:20:00 | ESP32_CAF_001 | Café      | Colheita/Frutificacao |              25.29 |             31.73 |      6.07 |               64.07 |            0 |            0 |            0 |              0 | CONDICAO IDEAL       |
| 2025-10-21 18:30:00 | ESP32_CAF_001 | Café      | Colheita/Frutificacao |              22.96 |             41.38 |      6.19 |               60.59 |            0 |            0 |            0 |              0 | CONDICAO IDEAL       |
| 2025-10-21 18:40:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              23.88 |             31.02 |      5.83 |               52.82 |            0 |            1 |            1 |              1 | CONDICAO IDEAL       |
| 2025-10-21 18:50:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              23.94 |             30.49 |      6.27 |               64.99 |            0 |            0 |            0 |              0 | CONDICAO IDEAL       |
| 2025-10-21 19:00:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              23.57 |             33.18 |      6.29 |               75.16 |            0 |            0 |            0 |              0 | MONITORAR            |
| 2025-10-21 19:10:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              20.92 |             44.81 |      6.45 |               53.78 |            0 |            1 |            0 |              0 | CONDICAO IDEAL       |
| 2025-10-21 19:20:00 | ESP32_CAF_001 | Café      | Colheita/Frutificacao |              23.62 |             46.66 |      5.5  |               61.34 |            0 |            0 |            0 |              0 | CONDICAO IDEAL       |
| 2025-10-21 19:30:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              22.37 |             39.09 |      5.33 |               48.57 |            0 |            0 |            0 |              1 | MONITORAR            |
| 2025-10-21 19:40:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              23.83 |             38.87 |      6.5  |               57.9  |            1 |            0 |            0 |              1 | CONDICAO IDEAL       |
| 2025-10-21 19:50:00 | ESP32_CAF_002 | Café      | Colheita/Frutificacao |              17.74 |             51.71 |      6.28 |               68.08 |            0 |            0 |            0 |              0 | CONDICAO IDEAL       |